In [6]:
import multiprocessing
import gensim
import sys
from random import shuffle



In [11]:
def parse_playlist_get_sequence(in_line, playlist_sequence):
    song_sequence = []
    #按行切分歌单文件，得到的一行表示一个歌单
    contents = in_line.strip().split("\t")
    # 解析歌单序列
    for song in contents[1:]:
        try:
            song_id, song_name, artist, popularity = song.split(":::")
            song_sequence.append(song_id)
        except:
            print("song format error")
            print(song+"\n")
        for i in range(len(song_sequence)):
            #打乱可加可不加，就是一个歌单有摇滚100首，word2vec认为第50首和窗口内的几首歌有联系，但是实际上第一首和第50首也有关系，打乱重复歌单
            #后就可以使的一个歌单内的所有歌曲都有均等机会经窗口的。不加的话影响也不大，可根据实际场景来决定。
            shuffle(song_sequence)
            playlist_sequence.append(song_sequence)


def train_song2vec(in_file, out_file):
    #所有歌单序列
    playlist_sequence = []
    #遍历所有歌单
    for line in open(in_file):
        parse_playlist_get_sequence(line, playlist_sequence)
    #使用word2vec训练
    cores = multiprocessing.cpu_count()
    print("Training word2vec model...")
    gensim.models.Word2Vec(sentences=playlist_sequence, size = 150, min_count=3, window= 7, workers=cores)
    print("Saving model...")
    model.save(out_file)


In [ ]:
song_squence_file = "./popular.playlist"
model_file = '/Users/wjj/Desktop/七月在线音乐推荐/recomendation_system_codes/song2vec.model'
#耗时比较久
train_song2vec(song_squence_file, model_file)


song format error
405599088:::Make Them Wheels Roll

song format error
:::SAFIA:::100.0

song format error
424496188:::大王叫我来巡山 - （原唱：

song format error
 贾乃亮/贾云馨）:::流浪的蛙蛙:::65.0

song format error
19169096:::

song format error
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

song format error
26902203:::What’s your name? (collaboration with 壇蜜)

song format error
:::SoulJa:::100.0

song format error
33054290:::

song format error
Heartbeats:::Dabin:::95.0

song format error
427373827:::Champions (From "Hands of Stone") 

song format error
:::Usher:::30.0

song format error
31654811:::

song format error
American Cowboys:::Tim Wynn:::65.0

song format error
19169096:::

song format error
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

song format error
19169096:::

song format error
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

Training word2vec model...


In [ ]:
#加载数据集和训练好的模型
import pickle
song_dic = pickle.load(open('populars_song.pkl','rb'))
model_str = './song2vec.model'
gensim.models.Word2Vec.load(model_str)


In [ ]:
#看一眼歌曲的id和mingzi
for song in song_dic.keys()[:10]:
    print(song,song_dic[song])
#字典的1000到1500里，每隔50取一首
song_id_list = song_dic.keys()[1000:1500:50]
for song_id in song_id_list:
    result_song_list = model.most_similar(song_id)
    print(song_id, song_dic[song_id])
    print('相似歌曲和相似度为：')
    for song in result_song_list:
        print('\t', song_dic[0],song[1])